In [9]:
import os
import pandas as pd
import re
import numpy as np
import collections
from operator import itemgetter
import pprint as pp
import re
import nltk
from nltk.corpus import stopwords
import nltk.data
import time


# #data from http://times.cs.uiuc.edu/~wang296/Data/
# files = os.listdir('./Review_Texts')
# #print(os.listdir('./Review_Texts'))
# print(len(files))

In [ ]:
cats = ['Rooms', 'Date', 'Location', 'Service', 'Business service', 'Author', 'Check in / front desk', 'No. Helpful', 'Cleanliness', 'Content', 'Value', 'No. Reader', 'Overall']
data = pd.DataFrame(columns=cats)

In [ ]:
def addFileToData(filename, data):
    intColumns = ['No. Reader', 'No. Helpful', 'Cleanliness','Check in / front desk', 'Value', 'Overall', 'Service', 'Business service', 'Rooms', 'Location']
    characterThreshold = 60
    with open(filename, 'r') as content_file:
        content = content_file.read()
        
    reviews = content.split("\n\n")
    for r in reviews:
        thisReview = pd.Series([None]*len(cats), cats)
        splt = r.split("\n")
        for s in splt:
            for c in cats:
                if "<"+c+">" in s:
                    value = s.replace('<'+c+'>', '')
                    if c in intColumns:
                        value = int(value)
                    if value == -1: #we dont want -1 as this is going to mess up averaging, take np.nan
                        value = np.nan

                    if c == "Content":
                        value = value.lower()

                    thisReview[c] = value
                    
        if not thisReview["Content"] == None and len(thisReview["Content"]) > characterThreshold:
            #only add if theres content and its long enough
            data = data.append(thisReview, ignore_index=True)
    return data
        
data = addFileToData('./Review_Texts/hotel_100504.dat', data) 
data = addFileToData('./Review_Texts/hotel_72572.dat', data) 


In [ ]:
print(data.shape)
data.head()

In [1]:
#tokenizer to split sentences
from nltk.tokenize import RegexpTokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer2 = RegexpTokenizer(r'\w+')

#Stemmer defined for stemming words
from nltk.stem.porter import *
stemmer = PorterStemmer()

NameError: name 'nltk' is not defined

# -----------------------------------------------------------------------------------------
#                                    FEATURE EXTRACTION: MODEL RUN
# -----------------------------------------------------------------------------------------

In [2]:
import proj_base
data = proj_base.getStandardData(numFiles=20)
print(data.shape)
data.head()

(2293, 13)


,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Dec 23, 2008",5.0,5.0,NaN,selizabethm,5.0,NaN,5.0,wonderful time- even with the snow! what a gre...,4.0,NaN,5.0
1,4.0,"Nov 13, 2008",5.0,5.0,NaN,IndieLady,5.0,NaN,4.0,"lovely hotel, unique decor, friendly front des...",5.0,NaN,4.0
2,4.0,"Nov 11, 2008",3.0,NaN,4,Hilobb,5.0,NaN,4.0,"nice hotel, expensive parking we got a good de...",4.0,NaN,4.0
3,5.0,"Nov 4, 2008",5.0,5.0,5,Chianti_girl24,5.0,NaN,5.0,fabulous hotel location and service are great....,5.0,NaN,5.0
4,NaN,"Oct 18, 2008",NaN,NaN,NaN,hothearted,NaN,2,NaN,"loved the monaco! staff was amazing, with a sm...",NaN,2,5.0


In [5]:
proj_base.aspectSegmentationBayes(data)
train = proj_base.getTrainingData(data)
print(train.shape)
train.head()

(582, 4)


,aspectRating,aspectSentences,Content,Overall
6,2.0,"[in comparison to the w, i spent about $45 mor...",nice rooms but not a 4* experience the hotel m...,3.0
7,5.0,[our daughter's 9th birthday was 09.13.08 and ...,wonderful experience at the monaco we are seat...,5.0
12,3.0,[the cost was a little high at $217 per night ...,a great hotel we stayed here in march as we we...,4.0
13,4.0,[a wonderful hotel at a reasonable price i jus...,a wonderful hotel at a reasonable price i just...,5.0
15,2.0,"[overall, i didn't have any problems with hote...",overpriced my husband and i stayed here the ni...,3.0


In [18]:
from nltk import bigrams
from nltk.util import ngrams
from sklearn.feature_extraction import DictVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
import string
from string import * 
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import numpy as np


#Stemmer defined for stemming words
from nltk.stem.porter import *
stemmer = PorterStemmer()


# print data['Content'][0]

#Function to clean reviews
def clean_string(review):
    sentence=review.lower()
    remove_list=stopwords.words("english")
    word_list=sentence.split()
    sentence=' '.join([i for i in word_list if i not in remove_list])
    translation=string.maketrans("","")
    clean_review = sentence.translate(translation,string.punctuation)
    return clean_review

# Clean all reviews of dataset and add to new list for feature creation
def Model_run(reviews_aspect_series,ratings_series):
    new_data_review=[]
    for r in reviews_aspect_series:
        concat=""
        for sentence in r:
            concat=concat+" " + sentence 
        new_data_review.append(concat)
    print new_data_review
#Obain unigram features (create a function for this)
#     unigram_vectorizer = CountVectorizer(min_df=1,binary=False)
#     X = unigram_vectorizer.fit_transform(new_data_review)
#     X_1=X.toarray()

# Obtain Bi-unigram features (create a function for this)
#     bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)
#     X_2 = bigram_vectorizer.fit_transform(new_data_review).toarray()
#     print len(X_2[0])

#Use tf-idf transformer on the unigram/bigram features (create function here)
#     transformer = TfidfTransformer(smooth_idf=False)
#     tfidf_X1 = transformer.fit_transform(X_1)
#     tfidf_X1.toarray()
#     tfidf_X2 = transformer.fit_transform(X_2)
#     tfidf_X2.toarray()
    tfidf_X1=proj_base.unigram_creation(new_data_review)
    tfidf_X2=proj_base.bigram_creation(new_data_review)


    Y=ratings_series
    np.array(Y)
    X_train, X_test, y_train, y_test = train_test_split(tfidf_X2, Y, test_size=0.2, random_state=0)
    clf = GaussianNB()
    clf.fit(X_train,y_train)
# clf.fit(tfidf[:160].toarray(), Y[:160])
# clf.score(X_test, y_test)

    params=clf.get_params(deep=True)
    print params

    sum1=0
    count=0
    for i in xrange(0,len(X_test)):
        a=clf.predict(X_test[i])
        b=np.array(y_test)[i]
        if a[0]== b:
            ind=1
        else:
            ind=0
        sum1=sum1+ind
        count=count+1
    return (sum1*1.0/count)

Model_run(train['aspectSentences'],train['aspectRating'])

[" in comparison to the w, i spent about $45 more per night but had a larger (square footage) room with a great soaking tub (with whirlpool jets) and a nice shower.before my stay, i had the hotel arrange a car service, the price $53 with tip was reasonable and the driver was waiting for me on arrival.checkin was easy but the downside was that with a room that was picked for its 2 person jacuzi tub, i found no bath accessories (salts, bubble bath etc...) and didn't get any during the stay.", " our daughter's 9th birthday was 09.13.08 and we promised to take her and two girlfriends to see shrek the musical at the 5th avenue theater, and decided we'd make a fun night out of it by booking a nearby hotel as well.we settled on the monaco based on its proximity to the theater, price, and trip adviser reviews, and couldn't have had a more rewarding experience.the play started at 2:00 pm, so the day before, i called and requested an early check-in of 1:00. i was told there was no guarantee, whi

C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Vinayak11\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  Deprec

0.4188034188034188

In [ ]:
#Get frequency distribution of cleaned words and select one with high frequency for unigrams and bigrams
new_data_review=[]
for i in data['Content']:
    new_data_review.append(clean_string(i))

word_list=[]
for i in new_data_review:
    temp=i.split()
    word_list.extend(temp)

word_freq = [word_list.count(w) for w in word_list]
freq_dict=dict(zip(word_list,word_freq))
sorted_list = sorted(freq_dict.items(), key=operator.itemgetter(1))

# take words with frequency > 1
lowfreq_word_list=[]
for pair in sorted_list:
    if pair[1]<10:
        lowfreq_word_list.append(pair[0])
    else:
        continue
#remove low frequency words from new_data_review

reviews_final=[]
for i in new_data_review:
    a=i.split()
    for j in a:
        if j in lowfreq_word_list:
            a.remove(j)
    sentence=' '.join([k for k in a])
    reviews_final.append(sentence)
print reviews_final
    


In [ ]:
Model_run(reviews_final,data['Overall'])

In [ ]:
# Test confusion matrix function
y_test=[5,1,2,4,1,2,3,5,5,1,3,3,4,1,2,3,4,5]
y_pred=[5,1,1,4,1,3,3,4,5,1,3,3,3,1,2,1,4,4]
class_names=['a','b','c','d','e']
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()